In [47]:
import pandas as pd
import numpy as np
import os

import urllib
from sqlalchemy import create_engine, inspect

DATABASES = {
  'default': {
     'ENGINE': 'mssql',
     'NAME': "Global_MATER",
     'USER': 'miebachindianapolis',
     "PASSWORD": 'Miebach1',
     "HOST": "miebachinternal.database.windows.net",
     "PORT": "1433",
     "OPTIONS": {"driver": "ODBC Driver 17 for SQL Server",
             },
     }
}

def connect_database():

    driver = DATABASES['default']['OPTIONS']['driver']
    server = DATABASES['default']['HOST']
    database = DATABASES['default']['NAME']
    username = DATABASES['default']['USER']
    password = DATABASES['default']['PASSWORD']

    # connection parameters
    params = 'DRIVER=' + driver + ';' \
        'SERVER=' + server + ';' \
        'DATABASE=' + database + ';' \
        'UID=' + username + ';' \
        'PWD=' + password + ';'

    conn_params = urllib.parse.quote_plus(params)
    dbconn = create_engine('mssql+pyodbc:///?odbc_connect=%s' %
                           conn_params, fast_executemany=False)

    autocommit_engine = dbconn.execution_options(isolation_level="AUTOCOMMIT")
    return dbconn
engine = connect_database()

print("Linked")

Linked


In [48]:
""" find out the projects with the wrong team and tasks """

wrong_projects_query = f'''
    select DISTINCT ph.project_id, ph.contract_holding_office_id as office_id
    from  global_manage_closeouttracker ct
    join global_manage_closeouttrackeritem cti on ct.id=cti.closeout_tracker_id
    join global_manage_closeoutteam cteam on ct.team_id=cteam.id
    join global_manage_closeoutteammember member on cteam.id=member.team_id
    join core_user on core_user.id=member.member_id
    join global_manage_projectmaster pm on ct.project_id=pm.id
    join global_manage_projectheader ph on pm.id=ph.project_id
    where core_user.office_id != ph.contract_holding_office_id
'''

wrong_projects_df = pd.read_sql(wrong_projects_query, engine)

print("wrong_projects_df - ", wrong_projects_df)

wrong_projects_df -      project_id  office_id
0          619         13
1          590         13
2          622         13
3          599         13
4          608         13
5          546         13
6          594         13
7          623         13
8          557         13
9          606         13
10         586         13
11         686         13
12         618         13
13         595         13
14         569         13
15         621         13
16         635         13
17         592         13


In [49]:
""" for each project, update or delete item data """

for index, row in wrong_projects_df.iterrows():
    project_id = row["project_id"]
    office_id = row['office_id']

    # print("project_id - ", project_id)
    # print("office_id - ", office_id)

    correct_task_query = f'''
        SELECT distinct task.id as correct_task_id, task.title as task_title
        FROM core_user member
        JOIN global_manage_closeoutteammember cotm ON cotm.member_id = member.id
        join global_manage_closeoutteam team on team.id=cotm.team_id
        join global_manage_closeouttask task on team.id=task.team_id
        WHERE member.office_id=${office_id}
    '''

    correct_task_df = pd.read_sql(correct_task_query, engine)
    # print("correct_task_df - ", correct_task_df)


    wrong_item_query = f'''
        select cti.id as item_id, cti.task_id as wrong_task_id, ctask.title as task_title
        from  global_manage_closeouttracker ct
        join global_manage_closeouttrackeritem cti on ct.id=cti.closeout_tracker_id
        join global_manage_closeoutteam cteam on ct.team_id=cteam.id
        join global_manage_closeouttask ctask on cti.task_id=ctask.id
        join global_manage_projectmaster pm on ct.project_id=pm.id
        join global_manage_projectheader ph on pm.id=ph.project_id
        where pm.id=${project_id}
    '''

    wrong_item_df = pd.read_sql(wrong_item_query, engine)
    # print("wrong_item_df - ", wrong_item_df)


    merged_item_df = pd.merge(wrong_item_df, correct_task_df, on='task_title', how='left')
    print("merged_item_df - ", merged_item_df)


    for index, row in merged_item_df.iterrows():
        item_id = row["item_id"]
        correct_task_id = row['correct_task_id']

        if pd.notna(correct_task_id):
            update_query = f"""
                UPDATE global_manage_closeouttrackeritem
                SET task_id = {correct_task_id}
                WHERE id = {item_id}
            """
            with engine.begin() as connection:
                connection.execute(update_query)
            print("updating ...")
        else:
            delete_query = f"""
                delete from global_manage_closeouttrackeritem
                WHERE id = {item_id}
            """
            with engine.begin() as connection:
                connection.execute(delete_query)
            print("deleting ...")

print("DONE")


merged_item_df -      item_id  wrong_task_id                            task_title  \
0      2673              1             All Project Fees Invoiced   
1      2756              2          All Travel Expenses Invoiced   
2      2839              3                      All Invoice Paid   
3      4153             19      All Subcontractors Invoices Paid   
4      2922              4           Feedback Sessions Completed   
5      3005              5                          3PL Database   
6      3088              6                          WMS Database   
7      3171              7       Project Documentation On Server   
8      3254              8           Project Case Study Complete   
9      3337              9  Contact List Saved In Project Folder   
10     3744             21                    Client Survey Sent   

    correct_task_id  
0              10.0  
1              11.0  
2              12.0  
3              20.0  
4               NaN  
5               NaN  
6          

In [50]:
""" for each project, update or delete tracker data """

for index, row in wrong_projects_df.iterrows():
    project_id = row["project_id"]
    office_id = row['office_id']

    correct_team_query = f'''
        SELECT distinct team.id as correct_team_id, team.title as team_title
        FROM core_user member
        JOIN global_manage_closeoutteammember cotm ON cotm.member_id = member.id
        join global_manage_closeoutteam team on team.id=cotm.team_id
        WHERE member.office_id=${office_id}
    '''

    correct_team_df = pd.read_sql(correct_team_query, engine)
    # print("correct_team_df - ", correct_team_df)

    wrong_tracker_query = f'''
        select ct.id as tracker_id, ct.team_id as wrong_team_id, cteam.title as team_title
        from  global_manage_closeouttracker ct
        join global_manage_closeoutteam cteam on ct.team_id=cteam.id
        join global_manage_projectmaster pm on ct.project_id=pm.id
        join global_manage_projectheader ph on pm.id=ph.project_id
        where pm.id=${project_id}
    '''

    wrong_tracker_df = pd.read_sql(wrong_tracker_query, engine)
    # print("wrong_tracker_df - ", wrong_tracker_df)


    merged_tracker_df = pd.merge(wrong_tracker_df, correct_team_df, on='team_title', how='left')
    print("merged_tracker_df - ", merged_tracker_df)

    for index, row in merged_tracker_df.iterrows():
        tracker_id = row["tracker_id"]
        correct_team_id = row['correct_team_id']

        if pd.notna(correct_team_id):
            update_query = f"""
                UPDATE global_manage_closeouttracker
                SET team_id = {correct_team_id}
                WHERE id = {tracker_id}
            """
            with engine.begin() as connection:
                connection.execute(update_query)
            print("updating ...")
        else:
            delete_query = f"""
                delete from global_manage_closeouttracker
                WHERE id = {tracker_id}
            """
            with engine.begin() as connection:
                connection.execute(delete_query)
            print("deleting ...")

print("DONE")

merged_tracker_df -     tracker_id  wrong_team_id            team_title  correct_team_id
0        1216              1             Financial              5.0
1        1299              2    People Development              6.0
2        1382              3  Knowledge Management              NaN
3        1465              4  Business development              7.0
updating ...
updating ...
deleting ...
updating ...
merged_tracker_df -     tracker_id  wrong_team_id            team_title  correct_team_id
0        1208              1             Financial              5.0
1        1291              2    People Development              6.0
2        1374              3  Knowledge Management              NaN
3        1457              4  Business development              7.0
updating ...
updating ...
deleting ...
updating ...
merged_tracker_df -     tracker_id  wrong_team_id            team_title  correct_team_id
0        1218              1             Financial              5.0
1        1301    

In [51]:
""" make sure there is no wrong data """

wrong_projects_query = f'''
    select DISTINCT ph.project_id, ph.contract_holding_office_id as office_id
    from  global_manage_closeouttracker ct
    join global_manage_closeouttrackeritem cti on ct.id=cti.closeout_tracker_id
    join global_manage_closeoutteam cteam on ct.team_id=cteam.id
    join global_manage_closeoutteammember member on cteam.id=member.team_id
    join core_user on core_user.id=member.member_id
    join global_manage_projectmaster pm on ct.project_id=pm.id
    join global_manage_projectheader ph on pm.id=ph.project_id
    where core_user.office_id != ph.contract_holding_office_id
'''

wrong_projects_df = pd.read_sql(wrong_projects_query, engine)

print("wrong_projects_df - ", wrong_projects_df)

print('Done')

wrong_projects_df -  Empty DataFrame
Columns: [project_id, office_id]
Index: []
Done
